Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [43]:
%run yelp_foursquare_EDA.ipynb

                                names  ratings
392  Stanley Park English Bay Seawall      9.6
391                      Stanley Park      9.6
127                 English Bay Beach      9.5
0                 33 Acres Brewing Co      9.4
370                   Seaside Seawall      9.4
214                   Kitsilano Beach      9.4
335              Queen Elizabeth Park      9.4
97               Coal Harbour Seawall      9.4
352                          Revolver      9.3
133               False Creek Seawall      9.3


In [88]:
#Joining the data
totalData = pd.DataFrame.merge(bikesDataFrame, fsquareDataFrame, how='inner', on=['latitude','longitude'])

       latitude   longitude  free_bikes                                names  \
0     49.262487 -123.114397          22                        Marulilu Cafe   
1     49.262487 -123.114397          22         La Taqueria Pinche Taco Shop   
2     49.262487 -123.114397          22                          Whole Foods   
3     49.262487 -123.114397          22              Elysian Coffee Broadway   
4     49.262487 -123.114397          22               Rogue Kitchen & Wetbar   
...         ...         ...         ...                                  ...   
2428  49.265442 -123.187738           9                   Browns Socialhouse   
2429  49.265442 -123.187738           9                    Yuji's From Japan   
2430  49.265442 -123.187738           9                     Beyond Bread Ltd   
2431  49.265442 -123.187738           9                 Banyen Books & Sound   
2432  49.265442 -123.187738           9  Aphrodite's Organic Cafe & Pie Shop   

      ratings                          

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [89]:
#Plotting and looking at some correlation coefficients for variables
import seaborn as sns
import matplotlib.pyplot as plt

trunTotal = totalData.drop(columns = ['names', 'locations'])
averages = trunTotal.groupby(['latitude', 'longitude']).mean()

print(averages)

sns.pairplot(averages)
plt.show()

print(averages['free_bikes'].corr(averages['ratings']))
print(averages['free_bikes'].corr(averages['distances']))
print(averages['free_bikes'].corr(averages['price']))
print(averages['free_bikes'].corr(averages['total_photos']))
print(averages['free_bikes'].corr(averages['popularity']))
print(averages['free_bikes'].corr(averages['total_ratings']))

sns.pairplot(trunTotal)
plt.show()

print(trunTotal['free_bikes'].corr(trunTotal['distances']))

0.11316238676939415
-0.04672836092336046
-0.0511027373541254
0.18782627814738276
0.09559270855468543
0.17624821579802158
0.028275043778948347
-0.037537179040521514


In [92]:
corrdict = {'free_bikes': [averages['free_bikes'].corr(averages['ratings']), 
                        averages['free_bikes'].corr(averages['distances']), 
                        averages['free_bikes'].corr(averages['price']), 
                        averages['free_bikes'].corr(averages['total_photos']),
                        averages['free_bikes'].corr(averages['popularity']),
                        averages['free_bikes'].corr(averages['total_ratings'])]}
corrdf = pd.DataFrame(data = corrdict, index=['ratings', 'distances', 'prices', 'total_photos', 'popularity', 'total_ratings'])
print(corrdf)

               free_bikes
ratings          0.113162
distances       -0.046728
prices          -0.051103
total_photos     0.187826
popularity       0.095593
total_ratings    0.176248


Generally speaking I do not see much of pattern in these graphs with regards to how many free bikes there are and other variables. The correlation coefficients generally show that as well as they are 
rather small values which indicates there is not much correlation. On the other hand popularity and total ratings do seem to be correlated based on the graphs which would make sense since the more people 
that go to place the more ratings will be written.

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [90]:
import sqlite3
from sqlite3 import Error
import math

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

connection = create_connection("sm_app.sqlite")

#Creating a table for the data
create_FourSquarePOI_table = """
CREATE TABLE IF NOT EXISTS FourSquarePOI (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  names TEXT NOT NULL,
  locations TEXT NOT NULL,
  free_bikes INTEGER,
  ratings NUMERIC NULL,
  latitude NUMERIC,
  longitude NUMERIC,
  price NUMERIC,
  popularity NUMERIC,
  total_ratings NUMERIC,
  total_photos NUMERIC,
  distance NUMERIC
);
"""

execute_query(connection, create_FourSquarePOI_table)  

#Inputting all the data from each row and making sure there are no NaNs but NULLs instead since sql can handle those
for index, row in totalData.iterrows():
    for key in row.keys():
        if type(row[key]) != str:
            if math.isnan(row[key]):
                row[key] = 'Null'
    create_data = """
    INSERT INTO
      FourSquarePOI (names, locations, free_bikes, ratings, latitude, longitude, price, popularity, total_ratings, total_photos, distance)
    VALUES
      (" """ + str(row["names"]) + """ ", '""" + str(row["locations"]) + "', "+ str(row["free_bikes"]) + ", "+ str(row["ratings"]) + ", "+ str(row["latitude"]) + ", "+ str(row["longitude"]) + ", "+ str(row["price"]) + """
      , """+ str(row["popularity"]) + ", " + str(row["total_ratings"]) + ", "+ str(row["total_photos"]) + ", "+ str(row["distances"]) + """); """
    
    execute_query(connection, create_data)

Connection to SQLite DB successful
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query execute

Look at the data before and after the join to validate your data.

In [14]:
print(totalData)
print(bikesDataFrame)
print(fsquareDataFrame)
#The dimensions make sense since they join to the largest data frame.

       latitude   longitude  free_bikes                                names  \
0     49.262487 -123.114397          21                        Marulilu Cafe   
1     49.262487 -123.114397          21         La Taqueria Pinche Taco Shop   
2     49.262487 -123.114397          21                          Whole Foods   
3     49.262487 -123.114397          21              Elysian Coffee Broadway   
4     49.262487 -123.114397          21               Rogue Kitchen & Wetbar   
...         ...         ...         ...                                  ...   
2428  49.265442 -123.187738           9                   Browns Socialhouse   
2429  49.265442 -123.187738           9                    Yuji's From Japan   
2430  49.265442 -123.187738           9                     Beyond Bread Ltd   
2431  49.265442 -123.187738           9                 Banyen Books & Sound   
2432  49.265442 -123.187738           9  Aphrodite's Organic Cafe & Pie Shop   

      ratings                          